In [1]:
from Workbench.transport.QuestClient import QuestDBClient

client = QuestDBClient(host="34.97.186.69",
                       port=8812,
                       read_only=True)

query ="""select * from top_of_book
where exchange = '{}' and symbol = '{}';
"""

df_htx = client.execute_query(query.format("HTX","SOL-USDT"))
df_binance = client.execute_query(query.format("Binance","SOLUSDT"))
df_bb = client.execute_query(query.format("Bybit","SOLUSDT"))

/Users/clarenceho/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
df_htx['mid_price'] = (df_htx['bid_price'] + df_htx['ask_price']) / 2
df_binance['mid_price'] = (df_binance['bid_price'] + df_binance['ask_price']) / 2
df_bb['mid_price'] = (df_bb['bid_price'] + df_bb['ask_price']) / 2
df_htx['spread_bp'] = (df_htx['ask_price'] - df_htx['bid_price']) / df_htx['mid_price'] * 10000
df_binance['spread_bp'] = (df_binance['ask_price'] - df_binance['bid_price']) / df_binance['mid_price'] * 10000
df_bb['spread_bp'] = (df_bb['ask_price'] - df_bb['bid_price']) / df_bb['mid_price'] * 10000

In [3]:
df_binance

,symbol,exchange,bid_price,bid_qty,ask_price,ask_qty,mid_price,spread_bp
timestamp,,,,,,,,
2025-05-25 14:32:27.082,SOLUSDT,Binance,171.88,6.04,171.89,1007.79,171.885,0.581784
2025-05-25 14:32:27.093,SOLUSDT,Binance,171.88,53.42,171.89,1007.79,171.885,0.581784
2025-05-25 14:32:27.239,SOLUSDT,Binance,171.88,53.42,171.89,1007.75,171.885,0.581784
2025-05-25 14:32:27.281,SOLUSDT,Binance,171.88,53.20,171.89,1007.75,171.885,0.581784
2025-05-25 14:32:27.412,SOLUSDT,Binance,171.88,52.19,171.89,1007.75,171.885,0.581784
...,...,...,...,...,...,...,...,...
2025-05-25 14:37:33.258,SOLUSDT,Binance,171.52,287.85,171.53,452.57,171.525,0.583005
2025-05-25 14:37:33.348,SOLUSDT,Binance,171.52,292.23,171.53,452.57,171.525,0.583005
2025-05-25 14:37:33.361,SOLUSDT,Binance,171.52,293.35,171.53,461.76,171.525,0.583005


In [4]:
df_merged = (
    df_htx[["mid_price", "spread_bp"]].rename(columns=lambda x: f"{x}_htx")
    .join(df_binance[["mid_price", "spread_bp"]].rename(columns=lambda x: f"{x}_binance"), how="outer")
    .join(df_bb[["mid_price", "spread_bp"]].rename(columns=lambda x: f"{x}_bb"), how="outer")
    .sort_index()
)

In [5]:

df_merged.fillna(method="ffill", inplace=True)

/var/folders/y8/5zd6qyks6gqgfm_tfrq0nb6w0000gn/T/ipykernel_91390/3103239665.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_merged.fillna(method="ffill", inplace=True)


In [6]:
df_merged

,mid_price_htx,spread_bp_htx,mid_price_binance,spread_bp_binance,mid_price_bb,spread_bp_bb
timestamp,,,,,,
2025-05-25 14:32:25.844,171.86525,0.005819,NaN,NaN,NaN,NaN
2025-05-25 14:32:25.944,171.86525,0.005819,NaN,NaN,NaN,NaN
2025-05-25 14:32:26.144,171.86525,0.005819,NaN,NaN,NaN,NaN
2025-05-25 14:32:26.244,171.86525,0.005819,NaN,NaN,NaN,NaN
2025-05-25 14:32:26.344,171.86525,0.005819,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-05-25 14:37:37.011,171.90980,-8.388120,171.525,0.583005,171.545,0.582937
2025-05-25 14:37:37.030,171.90980,-8.388120,171.525,0.583005,171.545,0.582937
2025-05-25 14:37:37.047,171.90980,-8.388120,171.525,0.583005,171.545,0.582937


In [7]:
#cal the diff from mid_price_htx to mid_price_binance to mid_price_bb
df_merged['mid_price_htx_binance_diff'] = df_merged['mid_price_htx'] - df_merged['mid_price_binance']
df_merged['mid_price_htx_bb_diff'] = df_merged['mid_price_htx'] - df_merged['mid_price_bb']
df_merged['mid_price_binance_bb_diff'] = df_merged['mid_price_binance'] - df_merged['mid_price_bb']


In [10]:
df_merged

,mid_price_htx,spread_bp_htx,mid_price_binance,spread_bp_binance,mid_price_bb,spread_bp_bb,mid_price_htx_binance_diff,mid_price_htx_bb_diff,mid_price_binance_bb_diff
timestamp,,,,,,,,,
2025-05-25 14:32:25.844,171.86525,0.005819,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-05-25 14:32:25.944,171.86525,0.005819,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-05-25 14:32:26.144,171.86525,0.005819,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-05-25 14:32:26.244,171.86525,0.005819,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-05-25 14:32:26.344,171.86525,0.005819,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2025-05-25 14:37:37.011,171.90980,-8.388120,171.525,0.583005,171.545,0.582937,0.3848,0.3648,-0.02
2025-05-25 14:37:37.030,171.90980,-8.388120,171.525,0.583005,171.545,0.582937,0.3848,0.3648,-0.02
2025-05-25 14:37:37.047,171.90980,-8.388120,171.525,0.583005,171.545,0.582937,0.3848,0.3648,-0.02
